# Assign noise to clusters

In [1]:
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from libpysal.graph import read_parquet
import numpy as np

In [ ]:
v = "v10"

model_params = "_post_processing_v1"

clusters_dir = "/data/uscuni-ulce/processed_data/clusters/"
chars_dir = "/data/uscuni-ulce/processed_data/chars/"
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"

In [6]:
cdata = pd.read_parquet(f"/data/uscuni-ulce/processed_data/clusters/cdata_{v}.pq")
# cdata = cdata.iloc[:, :-2]
cdata = cdata.drop("limLPS", axis=1)
cdata.index = cdata.index.astype(str)

In [32]:
def assign_noise(region_id, cdata):
    # read cluster data and characters
    clusters = pd.read_parquet(
        f"{clusters_dir}clusters_{region_id}_{v}.pq", columns=["final", "morph"]
    )
    chars = pd.read_parquet(f"{chars_dir}primary_chars_{region_id}.parquet")
    chars = chars[chars.index >= 0]
    assert (clusters.index == chars.index).all()

    # get morphotopes and noise groups values
    is_noise = clusters["final"].values == -1
    tq1 = read_parquet(graph_dir + f"tessellation_graph_{region_id}.parquet")
    noise_graph = tq1.subgraph(clusters[is_noise].index)
    components = noise_graph.component_labels
    noise_groups = chars[is_noise].groupby(components).median()

    # approax limAre
    noise_groups["limAre"] = (
        chars[is_noise]
        .groupby(components)["sdbAre"]
        .apply(lambda x: x.sort_values(ascending=True)[-min(30, x.shape[0]) :].sum())
    )

    noise_groups = noise_groups[cdata.columns]
    noise_groups.index = "-1_" + noise_groups.index.astype(str)

    all_data = pd.concat((cdata, noise_groups))
    scalar = StandardScaler()
    vals = scalar.fit_transform(all_data)
    all_data = pd.DataFrame(
        vals, index=all_data.index, columns=all_data.columns
    ).fillna(0)

    # setup tree data
    query_data = all_data.loc[noise_groups.index]
    tree_data = all_data.loc[cdata.index]

    ## query a knn classifier and assign predictions
    tree = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
    tree = tree.fit(X=tree_data.values, y=cdata.index.values)
    predictions = tree.predict(query_data.values)

    # map noise group clusters to building ids
    bid_to_ngroup = noise_graph.component_labels
    bid_to_prediction = bid_to_ngroup.map(
        pd.Series(
            predictions, query_data.index.str.split("_").str[-1].astype(int)
        ).to_dict()
    )
    clusters.loc[bid_to_prediction.index.values, "final"] = (
        bid_to_prediction.values.astype(int)
    )
    return clusters.final

In [ ]:
%%time

for region_id, _ in region_hulls.iterrows():
    print(region_id)
    final_without_noise = assign_noise(region_id, cdata)
    clusters = gpd.read_parquet(f"{clusters_dir}clusters_{region_id}_{v}.pq")
    # assignment hasnt changed existing clusters
    pd.testing.assert_series_equal(
        final_without_noise[clusters.final != -1], clusters.final[clusters.final != -1]
    )
    clusters["final_without_noise"] = final_without_noise
    clusters.to_parquet(f'{clusters_dir}clusters_{region_id}_{v}.pq')

4
10
132
134
286
313
400
523
765
801
832
913
960
1124
1154
1387
1478
1515
1605
1718
1736
1782
1970
1981
2096
2322
2350
2478
2514
2625
2728
2975
3039
3109
3150
3221
3250
3526
3610
3612
3701
3705
3752
3759
3981
4070
4214
4215
4235
4284
4356
4382
4723
4805
5096
5191
5246
5310
5408
5427
5662
5671
5766
5883
6254
6529
6560
6576
6741
6749
6811
6873
6996
7068
7094
7280
7485
7528
7534
7681
7688
7712
7727
7805
7914
7937
7963
8046
8216
8238
8256
8265
8345
8374
8396
8592
8707
8731
8757
8759
8813
9016
9064
9074
9150
9169
9194
9284
9824
9924
9954
9972
10019
10086
10095
10124
10179
10222
10263
10277
10455
10510
10511
10563
10579
10602
10666
10794
10847
10908
10926
10970
11002
11019
11057
11141
11210
11256
11261
11305
11309
11311
11318
11367
11444
11455
11471
11667
11678
11735
11757
11799
11877
11905
12027
12084
12100
12115
12154
12191
12381
12440
12483
12552
12667
12707
12755
12756
12844
12919
12965
13076
13137
13172
13191
13196
13229
13301
13395
13442
13482
13506
13553
13555
13614
13616
13655
13677
